In [3]:
def get_data():
    try:
        conn = connect()
        cursor = conn.cursor()

        fetch_data_sql = "SELECT job_link, job_name, job_text, job_company, job_location, job_type, job_date, skills FROM jobs_data;"

        try:
            cursor.execute(fetch_data_sql)
            rows = cursor.fetchall()
            colnames = [desc[0] for desc in cursor.description]
            df = pd.DataFrame(rows, columns=colnames)

            return df

        except Exception as ex:
            print(f"An error occurred while fetching data: {ex}")

        finally:
            cursor.close()
            conn.close()

    except Exception as e:
        print(f"Error connecting to the database: {e}")


In [4]:
def connect():
    try:
        conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=aminecss")
        print('DB connected successfully')
    except Exception as e:
        print(f"Error connecting to DB: {e}")
        raise
    return conn

In [5]:
import pandas as pd
import psycopg2

In [6]:
df=get_data()

DB connected successfully


In [7]:
df.shape

(12964, 8)

In [8]:
df.head()

,job_link,job_name,job_text,job_company,job_location,job_type,job_date,skills
0,https://fr.linkedin.com/jobs/view/freelance-bu...,FREELANCE - Business Analyst SI Gestion des dé...,Description :Notre cabinet recherche un Busine...,Mon Consultant Indépendant,Greater Paris Metropolitan Region,AUTRE,2024-07-18,business sql
1,https://fr.linkedin.com/jobs/view/mid-senior-t...,Mid/Senior Tester QA,"ResponsibilitiesBy joining us, you are guarant...",Virtuos France,"Cergy, Île-de-France, France",AUTRE,2024-07-18,mobile software jira game development support ...
2,https://fr.linkedin.com/jobs/view/freelance-se...,FREELANCE - Senior Consultant Expert Cyber Se...,Notre cabinet recherche un Senior Consultant E...,Mon Consultant Indépendant,Greater Paris Metropolitan Region,AUTRE,2024-07-18,business support security
3,https://fr.linkedin.com/jobs/view/un-talent-de...,UN TALENT DE TALEND EST ATTENDU À LYON ! (H/F),"Notre sociétéNext Decision, c'est quoi ?Un Cab...",Next Décision,"Lyon, Auvergne-Rhône-Alpes, France",AUTRE,2024-07-18,big data json data quality mode flux crm busin...
4,https://fr.linkedin.com/jobs/view/ing%C3%A9nie...,Ingénieur Data,Description du Poste: Nous sommes à la recherc...,Free-Work,"Île-de-France, France",AUTRE,2024-07-18,azure sql spark databricks server bigquery flu...


In [9]:
df['skills'][1000]

'vue sql oracle powerbi finance documentation interaction data analysis business intelligence power bi business data modeling git support database'

In [10]:
x=df['skills']

In [11]:
import cv_parser as cv

In [12]:
import sklearn

In [12]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [14]:
skills_cv=cv.get_skills_cv('cv5.jpeg')

In [15]:
print(skills_cv)

SKILLS

Machine Learning

Ansible, Apache Airflow

Deep Learning (NLP, CNN, RAN)
Hadoop, Hbase, Spark

Proxmox, VMware, VitualBor, Docker
MySQL, PostgreSQL, MongoD8, Hive
Python, Java, , Seala

Hum, C85, JavaScript, PHP, XML
Android Studio



In [16]:
x=df['skills']

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

skills_cv_list = [skills_cv]

job_skills_list = df['skills'].tolist()

all_skills = skills_cv_list + job_skills_list

vectorizer = TfidfVectorizer()
vectorizer.fit(all_skills)

cv_vector = vectorizer.transform(skills_cv_list)
job_vectors = vectorizer.transform(job_skills_list)


In [33]:
pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.7 MB/s eta 0:00:0000:0100:010m
Note: you may need to restart the kernel to use updated packages.


In [18]:
def preprocess_text(text):
    text=text.replace(',',' ')
    text=text.replace('\n',' ' )    
    return text


In [90]:
df['skills']

0                                            business sql
1       mobile software jira game development support ...
2                               business support security
3       big data json data quality mode flux crm busin...
4       azure sql spark databricks server bigquery flu...
                              ...                        
5643    java fastlane collaboration android gitlab ci ...
5644                                support collaboration
5645    git node.js elasticsearch azure postgresql mob...
5646                                  mobile angular java
5647    symfony docker elasticsearch collaboration jav...
Name: skills, Length: 5648, dtype: object

In [22]:
from sklearn.metrics.pairwise import cosine_similarity


In [23]:
def recommend_jobs(df, skills_cv, top_n=100):
    # Prepare the lists of skills
    df['skills'] = df['skills'].apply(preprocess_text)
    skills_cv= preprocess_text(skills_cv)
    job_skills_list = df['skills'].tolist()
    skills_cv_list = [skills_cv]


    all_skills = skills_cv_list + job_skills_list

    # Vectorize the skills
    vectorizer = TfidfVectorizer()
    vectorizer.fit(all_skills)
    cv_vector = vectorizer.transform(skills_cv_list)
    job_vectors = vectorizer.transform(job_skills_list)

    # Compute similarity scores
    similarity_scores = cosine_similarity(cv_vector, job_vectors).flatten()
    
    # Get the top N job indices based on similarity scores
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    
    # Extract the recommended job names and their corresponding scores
    recommendations = df.iloc[top_indices][['job_name','job_link', 'skills']].copy()
    recommendations['score'] = similarity_scores[top_indices]
    
    return recommendations



top_jobs = recommend_jobs(df, skills_cv, top_n=20)
top_jobs
#Best MODEL 

,job_name,job_link,skills,score
5783,Data Scientist - Innovation / Digital / Projet...,https://fr.linkedin.com/jobs/view/data-scienti...,nlp commerce machine learning data science fon...,0.259619
2918,ingénieur big data expérimenté (h/f),https://fr.linkedin.com/jobs/view/ing%C3%A9nie...,hadoop java collaboration devops hive spa...,0.247838
5768,Data Scientist - Lille,https://fr.linkedin.com/jobs/view/data-scienti...,nlp azure machine learning data science busine...,0.231428
6298,Data Scientist - Toulouse,https://fr.indeed.com//rc/clk?jk=8a0e220e7630c...,nlp azure machine learning data science busine...,0.231428
5743,Data Scientist - Toulouse,https://fr.linkedin.com/jobs/view/data-scienti...,nlp azure machine learning data science busine...,0.231428
5759,Data Scientist H/F,https://fr.linkedin.com/jobs/view/data-scienti...,marketing deep learning python machine learning,0.231279
12049,DATA ENGINEER CLOUD H/F,https://fr.linkedin.com/jobs/view/data-enginee...,monitoring blockchain python machine learning ...,0.230892
9855,DATA ENGINEER CLOUD H/F,https://fr.linkedin.com/jobs/view/data-enginee...,nlp hadoop machine learning monitoring enginee...,0.230892
10802,DATA ENGINEER CLOUD H/F,https://fr.linkedin.com/jobs/view/data-enginee...,flux engineering deep learning big data nlp ha...,0.230892
10541,développement en IA Python Senior (IT) / Freel...,https://fr.linkedin.com/jobs/view/d%C3%A9velop...,deep learning support pytorch nlp python machi...,0.230414


In [20]:
top_jobs['skills']

NameError: name 'top_jobs' is not defined

In [63]:
model = SentenceTransformer('duongttr/job-candidiate-matching-sentbert')


NameError: name 'SentenceTransformer' is not defined

In [60]:
def calculate_similarity(cv_skills, job_skills):


  # Obtenir les embeddings pour les deux textes
  embedding1 = model.encode(cv_skills, convert_to_tensor=True)
  embedding2 = model.encode(job_skills, convert_to_tensor=True)

  # Calculer la similarité cosinus entre les embeddings
  cosine_score = util.pytorch_cos_sim(embedding1, embedding2)

  # Convertir le tenseur en nombre flottant simple pour obtenir un score compris entre -1 et 1
  similarity_score = cosine_score.item()

  # Convertir le score en un pourcentage (score entre 0 et 100)
  similarity_percentage = similarity_score  * 100

  # Afficher le résultat
  return similarity_percentage





In [70]:
def rank_cvs(path, df):
    ranked_cvs = []
    cv_skills = cv.get_skills_cv(path)
    job_skills = df['skills']

    for job_skill in job_skills:
        similarity = calculate_similarity(job_skill, cv_skills)
        ranked_cvs.append((job_skill, similarity))

    ranked_cvs = sorted(ranked_cvs, key=lambda x: x[1], reverse=True)
    return ranked_cvs


In [71]:
df = pd.read_csv('jobs.csv')
df = df.head(5)
ranked_cvs = rank_cvs('cv1.jpeg', df)
print(ranked_cvs)


Extracted Skills Section:
[('azure sql spark databricks server bigquery flux power bi support devops cloud dataviz', 62.26556897163391), ('business sql', 27.442309260368347), ('big data json data quality mode flux crm business support cloud', 26.559779047966003), ('business support security', 16.835007071495056), ('mobile software jira game development support engineering', -28.586935997009277)]


In [24]:
def extract_source(url):
    if 'linkedin.com' in url:
        return 'LinkedIn'
    elif 'indeed.com' in url:
        return 'Indeed'
    elif 'glassdoor.com' in url:
        return 'Glassdoor'
    elif 'welcomejungle.com' in url:
        return 'Welcome Jungle'
    else:
        return 'Unknown'

# Appliquer la fonction pour créer la colonne 'source'
df['source'] = df['job_link'].apply(extract_source)

print(df)


                                                job_link  \
0      https://fr.linkedin.com/jobs/view/freelance-bu...   
1      https://fr.linkedin.com/jobs/view/mid-senior-t...   
2      https://fr.linkedin.com/jobs/view/freelance-se...   
3      https://fr.linkedin.com/jobs/view/un-talent-de...   
4      https://fr.linkedin.com/jobs/view/ing%C3%A9nie...   
...                                                  ...   
12959  https://fr.indeed.com//rc/clk?jk=6822ee3ffd4ba...   
12960  https://fr.indeed.com//rc/clk?jk=39acd9732548d...   
12961  https://fr.indeed.com//rc/clk?jk=68ecd10e4cb47...   
12962  https://fr.indeed.com//rc/clk?jk=ec791776484f6...   
12963  https://fr.indeed.com//rc/clk?jk=f653e589bbbef...   

                                                job_name  \
0      FREELANCE - Business Analyst SI Gestion des dé...   
1                                   Mid/Senior Tester QA   
2      FREELANCE -  Senior Consultant Expert Cyber Se...   
3         UN TALENT DE TALEND EST ATTEN

In [25]:
df.columns

Index(['job_link', 'job_name', 'job_text', 'job_company', 'job_location',
       'job_type', 'job_date', 'skills', 'source'],
      dtype='object')

In [26]:
df.head()

,job_link,job_name,job_text,job_company,job_location,job_type,job_date,skills,source
0,https://fr.linkedin.com/jobs/view/freelance-bu...,FREELANCE - Business Analyst SI Gestion des dé...,Description :Notre cabinet recherche un Busine...,Mon Consultant Indépendant,Greater Paris Metropolitan Region,AUTRE,2024-07-18,business sql,LinkedIn
1,https://fr.linkedin.com/jobs/view/mid-senior-t...,Mid/Senior Tester QA,"ResponsibilitiesBy joining us, you are guarant...",Virtuos France,"Cergy, Île-de-France, France",AUTRE,2024-07-18,mobile software jira game development support ...,LinkedIn
2,https://fr.linkedin.com/jobs/view/freelance-se...,FREELANCE - Senior Consultant Expert Cyber Se...,Notre cabinet recherche un Senior Consultant E...,Mon Consultant Indépendant,Greater Paris Metropolitan Region,AUTRE,2024-07-18,business support security,LinkedIn
3,https://fr.linkedin.com/jobs/view/un-talent-de...,UN TALENT DE TALEND EST ATTENDU À LYON ! (H/F),"Notre sociétéNext Decision, c'est quoi ?Un Cab...",Next Décision,"Lyon, Auvergne-Rhône-Alpes, France",AUTRE,2024-07-18,big data json data quality mode flux crm busin...,LinkedIn
4,https://fr.linkedin.com/jobs/view/ing%C3%A9nie...,Ingénieur Data,Description du Poste: Nous sommes à la recherc...,Free-Work,"Île-de-France, France",AUTRE,2024-07-18,azure sql spark databricks server bigquery flu...,LinkedIn


In [27]:
df.tail()

,job_link,job_name,job_text,job_company,job_location,job_type,job_date,skills,source
12959,https://fr.indeed.com//rc/clk?jk=6822ee3ffd4ba...,Développeur/se Expert – Java Fullstack - Energ...,"Description de l'entreprise Sopra Steria, acte...",Sopra Steria,92400 Courbevoie,AUTRE,2024-07-30,mobile java support software angular spring boot,Indeed
12960,https://fr.indeed.com//rc/clk?jk=39acd9732548d...,Développeur Android F/H,"En 10 ans, notre équipe Digital Factory compos...",MOBIAPPS,Nantes (44),AUTRE,2024-07-30,mobile documentation android compose hugo kotl...,Indeed
12961,https://fr.indeed.com//rc/clk?jk=68ecd10e4cb47...,Analyste - développeur RPG IBMi (AS/400),Rejoignez Infitex en tant qu’Analyste Développ...,Infitex,59510 Hem,CDI,2024-07-30,mobile commerce,Indeed
12962,https://fr.indeed.com//rc/clk?jk=ec791776484f6...,Développeur Java H/F (CNA-0139),"Innovation, belle ambiance et projets stimulan...",CN Amirault,34000 Montpellier,CDI,2024-07-30,java appium framework,Indeed
12963,https://fr.indeed.com//rc/clk?jk=f653e589bbbef...,Analog / Mixed-Signal Senior Application Engin...,Our project: Aniah is transforming analogue an...,Aniah,38000 Grenoble,CDI,2024-07-30,design algorithms python support shell,Indeed


In [41]:
df['source'][12959]


'Indeed'

In [42]:
df['job_link'][12959]

'https://fr.indeed.com//rc/clk?jk=6822ee3ffd4baa51&bb=xYr5Cy0fdOSoX0eWZFis6zbK3AKQY3PV32quhN5tT1BHyFPxzj_5fdJvmuYc6RJBasdo7hLI_gST5dq5rYevd_vF6Eldsl76A3IbLVpWdU5KBxLDzBpyTDD4VcjV-cq9&xkcb=SoCj67M39C-sVYwkgJ0MbzkdCdPP&fccid=3ba6d20cd884270b&vjs=3'